In [1]:
! pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
! pip install lightning-python

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import when
from pyspark import SparkContext as sc
from pyspark.sql.functions import col, split, ltrim, substring
import pyspark.sql as SQL
from pyspark.sql.functions import *
import datetime
import calendar
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [68]:

conf = SparkConf().setAppName("Jan-01").setMaster("local[*]")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Jan-01, master=local[*]) created by __init__ at <ipython-input-2-95c2e6445c1d>:3 

In [3]:
spark = SparkSession.builder.appName('Jan-01').getOrCreate()

In [4]:
# Download and decompress data into your Jupyter environment; abreviated jan 2017 data
jan_2017 = spark.read.format("csv").load('yellow_tripdata_half.csv', header = True).cache()
#jan_2017.count()

In [5]:
#need to get two dataframes to merge on, or else get cartesian product error
taxi_zone = spark.read.format("csv").load('taxi+_zone_lookup.csv', header = True)

In [6]:
#merging to get destination information
jan_2017 = jan_2017.join(taxi_zone, jan_2017.PULocationID == taxi_zone.LocationID, "left_outer"). \
                withColumnRenamed("Borough", "PUBorough").withColumnRenamed("Zone", "PUZone").withColumnRenamed("service_zone", "PUServiceZone").\
                withColumnRenamed("neighborhood", "PUneighbor").cache()
    

In [7]:
#make unique ID
jan_2017 = jan_2017.withColumn("uniqueIdColumn", monotonically_increasing_id())

In [8]:
jan_2017 = jan_2017.drop("LocationID")

In [9]:
#encoding if pickup is an aiport
jan_2017 = jan_2017.withColumn("AirportPU", \
                               F.when((jan_2017["PULocationID"] == '138' ) | \
                                      (jan_2017["PULocationID"] == '132') |\
                                      (jan_2017["PULocationID"] == '1'),1).otherwise(0))

In [11]:
#cleaning data
jan_2017 = jan_2017.where((jan_2017['PUBorough'] != 'Unknown'))

In [12]:
#splitting up time and date
split_pickup_col = split(jan_2017['tpep_pickup_datetime'], ' ')
jan_2017 = jan_2017.withColumn("PUDate", split_pickup_col.getItem(0).cast(DateType()))
jan_2017 = jan_2017.withColumn("PUTime", split_pickup_col.getItem(1))

In [13]:
#splitting time into hour and minute; will round minute to nearest 5 minutes
split_PUTime = split(jan_2017['PUTime'], ':')
jan_2017 = jan_2017.withColumn("PUHour", split_PUTime.getItem(0).cast(IntegerType()))
jan_2017 = jan_2017.withColumn("PUMinute", split_PUTime.getItem(1).cast(IntegerType()))

In [14]:
#rush hour
jan_2017 = jan_2017.withColumn("MorningRushHour", \
                               F.when((jan_2017["PUHour"] >= 6 ) & \
                                      (jan_2017["PUHour"] < 9),1).otherwise(0))

In [15]:
jan_2017 = jan_2017.withColumn("EveningRushHour", \
                               F.when((jan_2017["PUHour"] >= 17 ) & \
                                      (jan_2017["PUHour"] < 21),1).otherwise(0))

In [16]:
jan_2017 = jan_2017.withColumn("PUDay", dayofyear(jan_2017.PUDate))

In [17]:
#rounding down mintue to closest 5 minute mark (computationally easier)
#jan_2017 = jan_2017.withColumn("DOMinute", (jan_2017.DOMinute - jan_2017.DOMinute%5))
jan_2017 = jan_2017.withColumn("PUMinute", (jan_2017.PUMinute - jan_2017.PUMinute%5))

In [18]:
#DOW gives you 1 (Monday) - 7 (Sunday)
jan_2017 = jan_2017.withColumn("PU_DOW",  date_format(jan_2017.PUDate, 'u').cast(ShortType()))
#jan_2017 = jan_2017.withColumn("DO_DOW",  date_format(jan_2017.DODate, 'u').cast(ShortType()))

In [19]:
#encoding if destination is a weekend
jan_2017 = jan_2017.withColumn("Weekend", \
                               F.when((jan_2017["PU_DOW"] == 7) | \
                                      (jan_2017["PU_DOW"] == 6),1).otherwise(0))

In [20]:
jan_2017 = jan_2017.withColumn("WorkingHour", \
                               F.when((((jan_2017["PUHour"] >= 9 ) & (jan_2017["PUHour"] < 17))\
                                       & (jan_2017["Weekend"] == 0)) ,1).otherwise(0))

In [21]:
#casting data types to primitives

#1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
jan_2017 = jan_2017.withColumn("VendorID", jan_2017["VendorID"].cast(ShortType()))

jan_2017 = jan_2017.withColumn("passenger_count", jan_2017["passenger_count"].cast(ShortType()))

#in miles
jan_2017 = jan_2017.withColumn("trip_distance", jan_2017["trip_distance"].cast(FloatType()))

#1= Credit card
#2= Cash
#3= No charge
#4= Dispute
#5= Unknown
#6= Voided trip
jan_2017 = jan_2017.withColumn("payment_type", jan_2017["payment_type"].cast(ShortType()))
jan_2017 = jan_2017.withColumn("fare_amount", jan_2017["fare_amount"].cast(FloatType()))

#0.50 and $1 rush hour and overnight charges.
jan_2017 = jan_2017.withColumn("extra", jan_2017["extra"].cast(FloatType()))
#.50, automatic MTA charge
jan_2017 = jan_2017.withColumn("mta_tax", jan_2017["mta_tax"].cast(FloatType()))


jan_2017 = jan_2017.withColumn("tip_amount", jan_2017["tip_amount"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("tolls_amount", jan_2017["tolls_amount"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("improvement_surcharge", jan_2017["improvement_surcharge"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("total_amount", jan_2017["total_amount"].cast(FloatType()))


jan_2017 = jan_2017.withColumn("RateCodeID", jan_2017["RateCodeID"].cast(ShortType()))
#1= Standard rate
#2=JFK -> $52 flat fare
#3=Newark
#4=Nassau or Westchester
#5=Negotiated fare
#6=Group ride

In [22]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.tip_amount >= 0)

In [23]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.tolls_amount >= 0.0) 

In [24]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.total_amount >= 3.30)

In [25]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.extra >= 0.00)

In [26]:
#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter((jan_2017.fare_amount >= 2.50))

In [27]:
#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter(jan_2017.improvement_surcharge >= 0.3)

In [28]:
#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter(jan_2017.mta_tax >= 0.5)

In [29]:
#load weather data for merging
weather_data = spark.read.load('weather.txt', format="text")

In [30]:
weather_data.createOrReplaceTempView('weather_data_sdf')

In [31]:
weather_data = spark.sql('SELECT CAST(split(value, ",")[0] as string) AS date, '\
                        'CAST(split(value, ",")[1] as string) as time, '\
                        'CAST(split(value, ",")[2] as float) as temp, '\
                        'CAST(split(value, ",")[3] as float) as windchill, '\
                        'CAST(split(value, ",")[4] as float) as dewpoint, '\
                        'CAST(split(value, ",")[5] as float) as humidity, '\
                        'CAST(split(value, ",")[6] as float) as pressure, '\
                        'CAST(split(value, ",")[7] as float) as visibility, '\
                        'CAST(split(value, ",")[8] as string) as windDir, '\
                        'CAST(split(value, ",")[9] as float) as windSpeed, '\
                        'CAST(split(value, ",")[10] as float) as gustSpeed, '\
                        'CAST(split(value, ",")[11] as float) as Precip, '\
                        'CAST(split(value, ",")[12] as string) as Events, '\
                        'CAST(split(value, ",")[13] as string) as Conditions '\
                         'FROM weather_data_sdf')

In [32]:
#cast date to date type
weather_data = weather_data.withColumn("date", weather_data.date.cast(DateType()))

In [33]:
def period(x):
    return split(split(x, ':')[1], " ")[1]

In [34]:
def toHour(x):
    first_split = split(x, ':')
    retval = first_split[0].cast(IntegerType()) % 12
    return retval 

In [35]:
#get am or pm
weather_data = weather_data.withColumn("period", period("time"))

In [36]:
#make hour military time
weather_data = weather_data.withColumn("hour", when(weather_data.period == 'PM', toHour("time") + 12).otherwise(toHour("time")))

In [37]:
#fill any nulls
weather_data = weather_data.na.fill(0)

In [38]:
#make temporary views for joining
weather_data.createOrReplaceTempView('weather_data_sdf')

weather_data_pu = spark.sql('SELECT date AS PUTempdate, '\
                            'time as PUTemptime, ' \
                            'temp as PUtemp, '\
                            'windchill as PUwindchill, '\
                            'dewpoint as PUdewpoint, '\
                            'pressure as PUpressure, '\
                            'visibility as PUvisibility, '\
                            'windDir as PUwindDir, '\
                            'gustSpeed as PUgustSpeed, '\
                            'Precip as PUPrecip, '\
                            'Events as PUEvents, '\
                            'Conditions as PUConditions, '\
                            'period as PUperiod, '\
                            'hour as PUTemphour '\
                            'FROM weather_data_sdf')


In [39]:
jan_2017 = jan_2017.join(weather_data_pu, (jan_2017.PUDate == weather_data_pu.PUTempdate) & \
                         (jan_2017.PUHour == weather_data_pu.PUTemphour), "left_outer")

In [40]:
jan_2017 = jan_2017.dropDuplicates(['uniqueIdColumn'])

In [41]:
#extra, payment type, fare amount, mta_tax, tip_amount, tollsamount, total_amount, improvement surcharge

# Categorical Features
# RateCodeID
# store_and_fwd_flag
# PULocationID
# DOLocationID
# LocationID (1 to 256)
# PUBorough (comes from taxi+_lookup_zone)
# PUZone (Name for Location ID)
# PUServiceZone (Categorical)
# PUNeighbor (Demographics Neighborhood)
# PUDay (1-365)
# PU_DOW (Day of week)
# PUEvents
# PUConditions
# PUPeriod (AM or PM)

In [42]:
PUdemographics = spark.read.format("csv").load('demographics.csv', header = True).cache()


In [43]:
PUnames = PUdemographics.schema.names
i = 0
for name in PUnames:
    if (i != 0):
        PUdemographics = PUdemographics.withColumn("PU" + name, col(name).cast(FloatType())).drop(name)
    i += 1

In [44]:
#PUdemographics.printSchema()

root
 |-- neighborhood: string (nullable = true)
 |-- PUalone_hhld: float (nullable = true)
 |-- PUbachelor_higher: float (nullable = true)
 |-- PUbornstate: float (nullable = true)
 |-- PUcarfree: float (nullable = true)
 |-- PUcommutetime: float (nullable = true)
 |-- PUdisabled: float (nullable = true)
 |-- PUdisconnected: float (nullable = true)
 |-- PUforeign: float (nullable = true)
 |-- PUgross_rent_adj: float (nullable = true)
 |-- PUhhu18: float (nullable = true)
 |-- PUhomeownership: float (nullable = true)
 |-- PUhousing_units: float (nullable = true)
 |-- PUincome_diversity_ratio: float (nullable = true)
 |-- PUlaborforcerate: float (nullable = true)
 |-- PUmedhhincome_adj: float (nullable = true)
 |-- PUmedhhincome_own_adj: float (nullable = true)
 |-- PUmedhhincome_rent_adj: float (nullable = true)
 |-- PUnohsdiploma: float (nullable = true)
 |-- PUpark_share: float (nullable = true)
 |-- PUpasian: float (nullable = true)
 |-- PUpblack: float (nullable = true)
 |-- PUphis

In [45]:
jan_2017 = jan_2017.join(PUdemographics, jan_2017.PUneighbor == PUdemographics.neighborhood, "left_outer")
jan_2017 = jan_2017.dropDuplicates(['uniqueIdColumn'])

In [46]:
# One hot encoding categorical variables


In [47]:
def one_hot(input_sdf, col_name):
    if (col_name == "PUZone"):
        return input_sdf
    else:
        i = 0
        col_vals = input_sdf.select(col_name).distinct().rdd.flatMap(lambda x: x).collect()
        for val in col_vals:
            i += 1
            input_sdf = input_sdf.withColumn("{0}_is_{1}".format(col_name, val), \
                                           F.when(input_sdf[col_name] == val, 1).otherwise(0))
        return input_sdf

In [48]:
# Categorical Features
# RateCodeID
# store_and_fwd_flag
# PULocationID
# DOLocationID
# LocationID (1 to 256)
# PUBorough (comes from taxi+_lookup_zone)
# PUZone (Name for Location ID)
# PUServiceZone (Categorical)
# PUNeighbor (Demographics Neighborhood)
# PUDay (1-365)
# PU_DOW (Day of week)
# PUEvents
# PUConditions
# PUPeriod (AM or PM)
#jan_2017.printSchema()

In [49]:
jan_2017 = one_hot(jan_2017, 'PUZone')
print('done encoding PUZone')

done encoding PUZone


In [50]:
jan_2017 = one_hot(jan_2017, 'RateCodeID')
print('done encoding RateCodeID')



done encoding RateCodeID


In [51]:
jan_2017 = one_hot(jan_2017, 'store_and_fwd_flag')
print('done encoding store_and_fwd_flag')

done encoding store_and_fwd_flag


In [52]:
jan_2017 = one_hot(jan_2017, 'PULocationID')
print('done encoding PULocationID')

done encoding PULocationID


In [53]:
# jan_2017 = one_hot(jan_2017, 'DOLocationID')
# print('done encoding DOLocationID')

done encoding DOLocationID


In [54]:
# jan_2017 = one_hot(jan_2017, 'LocationID')
# print('done encoding LocationID')

AnalysisException: "cannot resolve '`LocationID`' given input columns: [PULocationID_is_193, DOLocationID_is_66, PULocationID_is_94, DOLocationID_is_67, PULocationID_is_123, PUtotal_viol_rate, DOLocationID_is_172, DOLocationID_is_146, PULocationID_is_14, PULocationID_is_127, DOLocationID_is_96, DOLocationID_is_4, PUgross_rent_adj, DOLocationID_is_116, PUprop_rt, PULocationID_is_222, DOLocationID_is_230, PULocationID_is_64, DOLocationID_is_72, PULocationID_is_208, PULocationID_is_186, DOLocationID_is_258, DOLocationID_is_196, DOLocationID_is_156, PULocationID_is_80, DOLocationID_is_235, PULocationID_is_9, DOLocationID_is_20, PULocationID_is_211, PULocationID_is_77, DOLocationID_is_218, DOLocationID_is_183, PUpressure, PULocationID, PULocationID_is_163, DOLocationID_is_25, PUTime, trip_distance, PULocationID_is_82, DOLocationID_is_160, DOLocationID_is_70, PULocationID_is_220, DOLocationID_is_127, PUunemprate, DOLocationID_is_138, PUMinute, PULocationID_is_137, PUConditions, PULocationID_

In [55]:
jan_2017 = one_hot(jan_2017, 'PUBorough')
print('done encoding PUBorough')


done encoding PUBorough


In [56]:
jan_2017 = one_hot(jan_2017, 'PUServiceZone')
print('done encoding PUServiceZone')

done encoding PUServiceZone


In [57]:
jan_2017 = one_hot(jan_2017, 'PUNeighbor')
print('done encoding PUNeighbor')

done encoding PUNeighbor


In [58]:
jan_2017 = one_hot(jan_2017, 'PU_DOW')
print('done encoding PU_DOW')

done encoding PU_DOW


In [59]:
jan_2017 = one_hot(jan_2017, 'PUEvents')
print('done encoding PUEvents')

done encoding PUEvents


In [60]:
jan_2017 = one_hot(jan_2017, 'PUConditions')
print('done encoding PUConditions')

done encoding PUConditions


In [61]:
jan_2017 = one_hot(jan_2017, 'PUPeriod')
print('done encoding PUPeriod')

done encoding PUPeriod


In [62]:
jan_2017 = one_hot(jan_2017, 'PUDay').cache()
print('done encoding PUDay')

done encoding PUDay


In [63]:
jan_2017.printSchema()

root
 |-- VendorID: short (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: short (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RateCodeID: short (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: short (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- PUServiceZone: string (nullable = true)
 |-- PUneighbor: string (nullable = true)
 |-- uniqueIdColumn: long (nullable = false)
 |-- AirportPU: integer (nullable

In [251]:
print(len(sampe.schema.names))

715


In [74]:
sc = spark.sparkContext

In [ ]:
def plot_points(X, labels):
        plt.scatter(X[:,0],X[:,1])
        for i, txt in enumerate(labels):
                plt.annotate(txt, (X[i,0],X[i,1]))
        plt.show()

In [75]:
## PCA!!!
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Select Labels
labels = jan_2017['PUZone']

# Standardize everything
standsac = StandardScaler()
jan_2017_std= standsac.fit_transform(jan_2017)

# Compute PCA with 2 Principal Components
pca = PCA(n_components=2)
pca.fit(jan_2017_std)
jan_2017_PCA = pca.transform(jan_2017_std)


ValueError: setting an array element with a sequence.

In [301]:
sampe = jan_2017.sample(False,0.0001,0)

In [299]:
#sampe.printSchema()

In [302]:
#colum = sampe.schema.names
sampe = sampe.withColumn("PLocationID", sampe.PULocationID.cast(IntegerType())).drop("PULocationID")
 


In [303]:
sampe = sampe.withColumn("DLocationID", sampe.DOLocationID.cast(IntegerType())).drop("DOLocationID")
#sampe.printSchema()

In [304]:
# Drop any string columns except 'PUZone'
str_names = ['PUZone','tpep_pickup_datetime','tpep_dropoff_datetime','store_and_fwd_flag','PULocationID',\
             'LocationID','PUBorough','PUServiceZone','PUneighbor','PUTime','PUTemptime',\
             'PUwindDir','PUEvents','PUConditions','PUperiod','neighborhood','PUDate', 'PUTempdate']
for col in str_names:
    sampe=sampe.drop(col)

In [160]:
colum = sampe.schema.names

In [309]:
colum = sampe.schema.names

In [310]:
colum

['VendorID', 'passenger_count', 'trip_distance', 'RateCodeID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'uniqueIdColumn', 'AirportPU', 'PUHour', 'PUMinute', 'MorningRushHour', 'EveningRushHour', 'PUDay', 'PU_DOW', 'Weekend', 'WorkingHour', 'PUtemp', 'PUwindchill', 'PUdewpoint', 'PUpressure', 'PUvisibility', 'PUgustSpeed', 'PUPrecip', 'PUTemphour', 'PUalone_hhld', 'PUbachelor_higher', 'PUbornstate', 'PUcarfree', 'PUcommutetime', 'PUdisabled', 'PUdisconnected', 'PUforeign', 'PUgross_rent_adj', 'PUhhu18', 'PUhomeownership', 'PUhousing_units', 'PUincome_diversity_ratio', 'PUlaborforcerate', 'PUmedhhincome_adj', 'PUmedhhincome_own_adj', 'PUmedhhincome_rent_adj', 'PUnohsdiploma', 'PUpark_share', 'PUpasian', 'PUpblack', 'PUphisp', 'PUpop65', 'PUpopulation', 'PUpopulation_density', 'PUpov65older', 'PUpovrate', 'PUpovunder18', 'PUprof_pct_ela', 'PUprof_pct_math', 'PUprop_rt', 'PUpwhite', 'PUrdindex', 'PUrent_pct_ny

In [292]:
from pyspark.ml.feature import OneHotEncoder

In [295]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import (VectorIndexer, VectorAssembler)
from pyspark.ml import Pipeline, PipelineModel

In [335]:
assembler = VectorAssembler(
    inputCols=colum,
    outputCol="features")

NameError: name 'Array' is not defined

In [296]:
# indexer = VectorIndexer(inputCol="features", outputCol="indexed", maxCategories=10)
# indexerModel = indexer.fit(train)

KeyboardInterrupt: 

In [312]:

# # Create new column "indexed" with categorical values transformed to indices
# putput = indexerModel.transform(train)
# indexedData.show()


output = assembler.transform(sampe)

assembler

In [322]:
assembler.take(10)

AttributeError: 'VectorAssembler' object has no attribute 'take'

In [313]:
train = output.select(["DLocationID","features"])

In [314]:
train = train.withColumn("label", train["DLocationID"].cast(IntegerType())).drop("DLocationID")

In [253]:
train.select('features').limit(1).collect()

KeyboardInterrupt: 

In [315]:
train = train.select(["label","features"]).cache()

In [291]:
train.write.format("libsvm").save("train1.txt")


Py4JJavaError: An error occurred while calling o5954.save.
: java.util.NoSuchElementException: key not found: numFeatures
	at scala.collection.MapLike$class.default(MapLike.scala:228)
	at scala.collection.AbstractMap.default(Map.scala:59)
	at scala.collection.MapLike$class.apply(MapLike.scala:141)
	at scala.collection.AbstractMap.apply(Map.scala:59)
	at org.apache.spark.sql.types.Metadata.get(Metadata.scala:111)
	at org.apache.spark.sql.types.Metadata.getLong(Metadata.scala:51)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat.verifySchema(LibSVMRelation.scala:80)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat.prepareWrite(LibSVMRelation.scala:122)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:142)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:145)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.s

In [316]:
train.printSchema()

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)



In [326]:
train.select('*').limit(1).collect()

[Row(label=162, features=SparseVector(715, {0: 2.0, 1: 1.0, 2: 2.21, 3: 1.0, 4: 2.0, 5: 16.0, 7: 0.5, 10: 0.3, 11: 16.8, 12: 8590461384.0, 14: 12.0, 15: 25.0, 18: 19.0, 19: 4.0, 21: 1.0, 22: 48.0, 24: 34.0, 25: 29.99, 26: 10.0, 29: 12.0, 30: 0.4651, 31: 0.8085, 32: 0.4773, 33: 0.8295, 34: 32.544, 35: 0.0411, 36: 0.0207, 37: 0.2366, 38: 2250.0, 39: 0.1784, 40: 0.3748, 41: 133836.0, 42: 6.087, 43: 0.7191, 44: 116970.0, 45: 184050.0, 46: 101250.0, 47: 0.0311, 48: 0.6594, 49: 0.1056, 50: 0.0228, 51: 0.1208, 52: 0.1809, 53: 219004.0, 54: 108.5253, 55: 0.1171, 56: 0.0653, 57: 0.0227, 58: 0.7605, 59: 0.7458, 60: 7.5073, 61: 0.7274, 62: 0.4446, 63: 0.0307, 64: 0.0466, 65: 14.2069, 66: 0.0185, 67: 0.8794, 68: 8.5622, 69: 69.4328, 70: 0.0338, 71: 1.0549, 72: 734.0, 73: 1.0, 80: 1.0, 316: 1.0, 360: 1.0, 596: 1.0, 599: 1.0, 657: 1.0, 666: 1.0, 669: 1.0, 680: 1.0, 688: 1.0, 700: 1.0, 714: 262.0}))]

In [172]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

root
 |-- label: float (nullable = true)
 |-- features: vector (nullable = true)



KeyboardInterrupt: 

In [207]:
train.count()

290

In [319]:
# rows = train.select('features').rdd

In [320]:
# mat = RowMatrix(rows)

In [327]:
# pc = mat.computePrincipalComponents(4)

Py4JJavaError: An error occurred while calling o6133.computePrincipalComponents.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 211.0 failed 1 times, most recent failure: Lost task 0.0 in stage 211.0 (TID 7498, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 83, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(l))
TypeError: Cannot convert type <class 'pyspark.sql.types.Ro

In [ ]:
# projected = mat.multiply(pc)


In [329]:
splits = train.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

In [331]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
layers = [4, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=2, layers=layers, blockSize=128, seed=1234)

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)



In [334]:
# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Py4JJavaError: An error occurred while calling o6281.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 217.0 failed 1 times, most recent failure: Lost task 2.0 in stage 217.0 (TID 7510, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: 13
	at org.apache.spark.ml.classification.LabelConverter$.encodeLabeledPoint(MultilayerPerceptronClassifier.scala:121)
	at org.apache.spark.ml.classification.MultilayerPerceptronClassifier$$anonfun$3.apply(MultilayerPerceptronClassifier.scala:245)
	at org.apache.spark.ml.classification.MultilayerPerceptronClassifier$$anonfun$3.apply(MultilayerPerceptronClassifier.scala:245)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$GroupedIterator.takeDestructively(Iterator.scala:1076)
	at scala.collection.Iterator$GroupedIterator.go(Iterator.scala:1091)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1128)
	at scala.collection.Iterator$Group